In [1]:
import numpy as np
import sqlite3
from scipy.spatial import distance #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html


In [2]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select * from livs where "Huvudgrupp" = "Korv"')
for row in rows:
        result.append(row)

db_contents = np.array(result)

conn.close()

## Prototypiska i sin kategori
Syftet är att ta en huvudgrupp och se vad mittpunkten i den kategorin är och sen räkna ut vilka livsmedel som ligger närmast mittpunkten.

Varje näringsvärde blir en dimension, och varje livsmedel en punkt som har ett värde i alla de dimensionerna.

Medelvärdet av alla livsmedel blir en punkt som man kan utgå från när man räknar avstånd till alla livsmedlen, och när man har tillgång till avstånden så kan man ordna alla livsmedlen efter avståndet till mittpunkten.

Det är första steget. 

Andra steget är att se om man verkligen behöver alla näringsvärden för att ändå få samma ordning mellan livsmedlen. Vilka kan man ta bort?

### Utgångspunkter

Från Livsmedelsverkets data har vi valt en huvudgrupp, som "Grönsaker". Alla dess data ligger i db_contents, och den variabeln har en shape som säger hur många rader (livsmedel) och kolumner (namn, nummer, näringsvärden, klassificering) den innehåller.

In [3]:
db_contents.shape

(60, 63)

Kolumn 0 är namnet, 1 är numret och sen börjar näringsvärdena. 

columns är variabeln som innehåller de värden som används i beräkningarna. Under står de kolumner som är möjliga att använda. 

Vill man ha fram betydelsena av kolumnerna kan man göra något i stil med

```
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
columns = np.array(columnObject)
``` 

In [4]:
columns = np.array([ 2,  4,  5,  6,  7,  8, 9, 42, 50])
# Possible columns: [ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57]


dataset blir de kolumner som används. Från databasen brukar de komma som unicode, så det är lämpligt att konvertera till float.

In [5]:
dataset = db_contents[:,columns].astype(float)

Det är fantastiskt enkelt att räkna medelvärdet av varje kolumn med funktionen numpy.mean, som returnerar en array med lika många element som antal kolumner

Obs: axis behövs, för annars får man fel medelvärde...

In [6]:
means = np.mean(dataset, axis=0)

In [7]:
print(means, '\n', means.shape)

[ 248.20333333    5.95133333   19.69666667   12.14633333    0.2595
   59.1065        0.            4.83          1.40133333] 
 (9,)


Det är läskigt lätt även att räkna avståndet mellan alla punkterna i dataset och medelvärdena:

(Det är något med medelvärdena som gör att man måste sätta en extra klammer runt, för att dimensionaliteten ska stämma...)

In [8]:
dist = distance.cdist(dataset, [means], 'euclidean') #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

*dist* innehåller sen varje rads avstånd från medelvärdet i alla kolumner.

In [9]:
dist

array([[  98.06126744],
       [  46.59588213],
       [   8.39603806],
       [  12.49557208],
       [  34.88283229],
       [   8.39690747],
       [  16.50740203],
       [  11.10170055],
       [  11.7241171 ],
       [   8.74168872],
       [  15.78031015],
       [  10.05150677],
       [ 136.85984298],
       [ 105.88619152],
       [  56.1002031 ],
       [  26.56476843],
       [  31.03010294],
       [  45.24720384],
       [  12.90096851],
       [  69.5409792 ],
       [  25.4841916 ],
       [ 133.12165322],
       [  23.60694718],
       [  34.46215831],
       [   9.23442951],
       [  33.93019435],
       [  60.25758532],
       [  52.87824053],
       [  38.23159852],
       [ 126.29178712],
       [   6.73472507],
       [  21.35216746],
       [ 240.25559318],
       [  73.37032882],
       [  87.77792113],
       [  26.45192725],
       [  19.6822887 ],
       [  27.08210273],
       [  24.53297621],
       [  21.65847821],
       [  26.40170774],
       [  26.401

Jag har svårt att vänja mig vid det här med implicit sortering.

Först sorterar vi alltså värdena och får en "ordning", som vi sen använder som index för den array vi egentligen vill sortera...

In [10]:
order = np.argsort(dist.T[0])

In [11]:
order

array([30,  2,  5,  9, 58, 24, 59, 11,  7,  8,  3, 18, 10,  6, 36, 49, 31,
       39, 22, 43, 38, 42, 20, 40, 41, 35, 15, 48, 50, 37, 16, 25, 23,  4,
       51, 28, 47, 46, 44, 17,  1, 27, 14, 45, 26, 19, 33, 56, 52, 34, 54,
        0, 13, 55, 57, 53, 29, 21, 12, 32])

Eftersom vi har vår ursprungliga lista med oss i *db_contents* så kan vi skriva ut vilka kolumner som helst, och nu med raderna i rätt ordning utifrån medelvärdet i huvudgruppen.

Här tog det ett tag för mig att förstå hur jag skulle se på resultatet. Att se de första som näringsmässigt "lika" var inte så svårt, men hur är det med de livsmedel som hamnar sist i listan?

Jag tror att man ska se det som en (mångdimensionell) "periferi" där de sista i listan förstås är långt från centrum, men inte alls inbördes "lika". De kan ha stort avstånd till mitten i väldigt skilda dimensioner. Det enda som förenar dem är att de är udda... :-)

In [12]:
db_contents[order,0]

array(['Grillkorv fett 18%', 'Falukorv fett 19%',
       'Fläskkorv  fett ca 23% rå', 'Köttkorv fett ca 23% rå',
       'Ren korv rå', 'Isterband fett 15%', 'Lamm korv rå',
       'Medvurst fett ca 23% kokt', 'Frukostkorv fett 23%',
       'Rökt isterband fett ca19 %', 'Falukorv fett 23%',
       'Wienerkorv fett 23%', 'Köttkorv fett ca 21% kokt',
       'Fläskkorv fett ca 21%  kokt', 'Värmlandskorv fett ca 19% rå',
       'Korv typ falukorv stekt', 'Prinskorv fett 24%',
       'Falukorv mager stekt', 'Varmkorv fett 15%', 'Isterband stekt',
       'Frukostkorv fett 23% kokt', 'Frukostkorv stekt',
       'Skinkkorv fett 17%', 'Falukorv skivad kokt', 'Falukorv stekt',
       'Varmkorv konserv kylkonserv fett ca 19%', 'Varmkorv fett ca 22%',
       'Wienerkorv stekt', 'Wienerkorv kokt',
       'Värmlandskorv fett ca 18% kokt', 'Varmkorv fett 23% grillad kokt',
       'Rökt påläggskorv fett 10% ospec', 'Grillkorv fett 15%',
       'Falukorv fett 18%', 'Prinskorv stekt', 'Chorizo korv stekt

## Del två. Att jämföra ordningar...

Den här ordningen mellan livsmedel bygger på att man tar ett urval kolumner. Typiskt tar man så mycket information man har (= alla kolumner) för att få en mer relevant ordning.

Men kanske är det så att inte alla kolumner behövs för att ge den här ordningen? Vilka kolumner skulle man kunna ta bort?

Det enklaste är att kolla om ordningarna är identiska, så i nedanstående är det bara det som görs. 

Dags att summera ihop mycket av del 1 i en funktion. Utgå från db_contents som typ en global variabel. Skicka in kolumner till funktionen, så returneras en ordning mellan livsmedlen i db_contents/dataset

In [13]:
def get_order(columns):
    dataset = db_contents[:,columns].astype(float)
    means = np.mean(dataset, axis=0)
    dist = distance.cdist(dataset, [means], 'euclidean')
    order = np.argsort(dist.T[0])
    return order

Vi ska jämföra ursprungsordningen (order_0) med alla ordningar som uppkommer när man tar bort en kolumn. Vi kallar underordningarna för order_1 och löper igenom dem en i taget.

In [14]:
columns_0 = columns
order_0 = get_order(columns)

In [15]:
order_0

array([30,  2,  5,  9, 58, 24, 59, 11,  7,  8,  3, 18, 10,  6, 36, 49, 31,
       39, 22, 43, 38, 42, 20, 40, 41, 35, 15, 48, 50, 37, 16, 25, 23,  4,
       51, 28, 47, 46, 44, 17,  1, 27, 14, 45, 26, 19, 33, 56, 52, 34, 54,
        0, 13, 55, 57, 53, 29, 21, 12, 32])

Här är ett exempel på vad som händer när man tar bort en kolumn:

In [16]:
columns_1 = np.delete(columns,4)

In [17]:
columns_1

array([ 2,  4,  5,  6,  8,  9, 42, 50])

In [18]:
order_1 = get_order(columns_1)

In [19]:
x = np.array_equal(order_0, order_1)

In [20]:
x

True

In [21]:
columns_1

array([ 2,  4,  5,  6,  8,  9, 42, 50])

Om vi tar en uppsättning kolumner behöver vi loopa igenom alla uppsättningar som saknar en kolumn. Något i den här stilen:

In [22]:
size = columns.size
print(size)
a = np.empty([size,size-1],dtype=int)
#print(a)
for index, item in enumerate(columns):
    a[index]=np.delete(columns,index)
    print(a[index])

9
[ 4  5  6  7  8  9 42 50]
[ 2  5  6  7  8  9 42 50]
[ 2  4  6  7  8  9 42 50]
[ 2  4  5  7  8  9 42 50]
[ 2  4  5  6  8  9 42 50]
[ 2  4  5  6  7  9 42 50]
[ 2  4  5  6  7  8 42 50]
[ 2  4  5  6  7  8  9 50]
[ 2  4  5  6  7  8  9 42]


Om vi ska göra en funktion av det hela och få ut en array av de överflödiga kolumnerna:

In [23]:
def remove_dimensions (columns):
    columns_1 = np.empty([columns.size,columns.size-1],dtype=int)
    superfluous = []
    #print(a)
    for index, item in enumerate(columns):
        columns_1[index]=np.delete(columns,index)
        if np.array_equal(get_order(columns), get_order(columns_1[index])):
            superfluous.append(item)
    return superfluous

In [24]:
print (columns_0)
superfluous = remove_dimensions (columns_0)
superfluous

[ 2  4  5  6  7  8  9 42 50]


[7, 9]

In [25]:
columns

array([ 2,  4,  5,  6,  7,  8,  9, 42, 50])

dim_superfluous är den listan som växer i varje "ben" av rekursionen. När funktionen bottnar appendas den till superlist och kapas

superfluous är resultatet av remove_dimensions, dvs en eller flera överflödiga dimensioner. Att superfluous blir tom är det samma som att algoritmen bottnar

dim_orig är listan som varje varv skickas till rekursionen. Den kapas med den överflödiga dimensionen för varje varv.

In [26]:
dim_superfluous = []
#Rekursiv
def reduce (dim_orig, dim_superfluous ):
    print ('-> reduce: första raden', 'dim_orig:', dim_orig, 'dim_superfluous', dim_superfluous)
    superfluous = remove_dimensions (dim_orig)
    print ('vi beräknar superfluous till',superfluous)
    if superfluous == []:
        print ('<- bottnar')
        superlist.append(dim_superfluous[:])
        # https://stackoverflow.com/questions/40425554/python-shallow-copy-and-deep-copy-in-using-append-method?rq=1
        print ('superlist', superlist)
        return 
    else:
        for index, superfluou in enumerate(superfluous):
            print ('i for-loopen –', 'index:', index, 'superfluou:', superfluou, 'superfluous:', superfluous, )
            new_dim_superfluous = dim_superfluous[:]
            new_dim_superfluous.append(superfluou)
            print ('new_dim_superfluous efter lagt till superfluou', new_dim_superfluous)            
            new_dim_orig = dim_orig[:]
            new_dim_orig = np.delete(new_dim_orig, np.where(new_dim_orig==superfluou), 0)
            print ('dim_orig efter tagit bort superflou:', dim_orig)
            reduce (new_dim_orig, new_dim_superfluous)
        return 


In [27]:
superlist=[]
x = reduce (columns, [])
x

-> reduce: första raden dim_orig: [ 2  4  5  6  7  8  9 42 50] dim_superfluous []
vi beräknar superfluous till [7, 9]
i for-loopen – index: 0 superfluou: 7 superfluous: [7, 9]
new_dim_superfluous efter lagt till superfluou [7]
dim_orig efter tagit bort superflou: [ 2  4  5  6  7  8  9 42 50]
-> reduce: första raden dim_orig: [ 2  4  5  6  8  9 42 50] dim_superfluous [7]
vi beräknar superfluous till [9]
i for-loopen – index: 0 superfluou: 9 superfluous: [9]
new_dim_superfluous efter lagt till superfluou [7, 9]
dim_orig efter tagit bort superflou: [ 2  4  5  6  8  9 42 50]
-> reduce: första raden dim_orig: [ 2  4  5  6  8 42 50] dim_superfluous [7, 9]
vi beräknar superfluous till []
<- bottnar
superlist [[7, 9]]
i for-loopen – index: 1 superfluou: 9 superfluous: [7, 9]
new_dim_superfluous efter lagt till superfluou [9]
dim_orig efter tagit bort superflou: [ 2  4  5  6  7  8  9 42 50]
-> reduce: första raden dim_orig: [ 2  4  5  6  7  8 42 50] dim_superfluous [9]
vi beräknar superfluous t